# Running on QICK Hardware

This notebook shows how to use pynq-quantum with the **QICK** (Quantum Instrumentation Control Kit) backend on a real RFSoC board.

**Prerequisites:**
- A Xilinx RFSoC board (ZCU111, ZCU216, RFSoC4x2, etc.)
- The `qick` Python package installed
- A bitstream file (`.bit`) for your board
- Qubit frequencies calibrated for your hardware

> **Note:** This notebook is illustrative. The code below will not run without the QICK hardware and firmware connected.

In [ ]:
# Requires QICK hardware -- will raise ImportError or RuntimeError
# on machines without QICK firmware.

from pynq_quantum import QuantumOverlay, QubitController

# Load the QICK backend with a bitstream
# Replace the bitstream path with your actual .bit file
overlay = QuantumOverlay(
    backend="qick",
    bitstream="/home/xilinx/jupyter_notebooks/qick/qick.bit",
)

print(f"Backend: {overlay.backend_name}")
print(f"Capabilities: {overlay.backend.get_capabilities()}")

In [ ]:
# Configure qubit drive frequencies (values are hardware-specific)
ctrl = QubitController(overlay, num_qubits=2)
ctrl.set_qubit_frequency(0, 4.85e9)   # Qubit 0 at 4.85 GHz
ctrl.set_qubit_frequency(1, 5.02e9)   # Qubit 1 at 5.02 GHz

# Build and run a simple circuit on hardware
ctrl.h(0)
ctrl.cnot(0, 1)
ctrl.measure()
result = ctrl.run(shots=2000)

print(f"Hardware Bell state counts: {result.counts}")
print(f"Raw IQ data shape: {result.raw_data.shape if result.raw_data is not None else 'N/A'}")

overlay.close()

## Notes

- The QICK backend compiles gate operations into pulse instructions using `PulseCompiler`, then sends them to the RFSoC DAC/ADC channels.
- Qubit frequencies must be calibrated for your specific hardware. Typical superconducting qubit frequencies are in the 4--6 GHz range.
- Raw IQ (in-phase / quadrature) data is available via `result.raw_data` for custom analysis.
- Use `backend='auto'` in `QuantumOverlay` to auto-detect the board type from `/proc/device-tree/model` or the `PYNQ_BOARD` environment variable.